In [ ]:
#r "nuget: TorchSharp-cuda-windows"

using TorchSharp;
using static TorchSharp.TensorExtensionMethods;
using static TorchSharp.torch.distributions;

# Using cuda

This tutorial is the only one that does not use the 'TorchSharp-cpu' package. When you have a machine with a GPU that supports CUDA programming, you can use either 'TorchSharp-cuda-windows' or 'TorchSharp-cuda-linux' depending on your operating system. There is no CUDA distribution for MacOS. 

Using CUDA, especially for training, can boost the performance significantly, typically a couple of orders of magnitude. It may be the difference between model training being feasible and not.

Note: The tutorials won't require much in terms of capabilities, but for training real vision models with even modest data sizes, you need at least 8MB of dedicated GPU memory. Even something as simple as CIFAR10 (in the Examples solution in this repo) requires that much memory in order not to blow up. 6MB, a common memory size on Nvidia-enabled laptops, is not enough.



With the right backend package installed, using CUDA is very straight-forward. TorchSharp lets you create and use tensors on the GPU as easily as on the CPU. Previously, we haven't use the 'device' argument when creating tensors, but it's easy to use. Notice that the string representation looks difference -- it says 'cuda:0' instead of 'cpu' for the device.

In [ ]:
torch.ones(3,4, device: torch.CUDA)

If you have more than one GPU, you can target a specific one by creating a device object to represent each one. The static `torch.CUDA` instance defaults to the first enumerated GPU.

Using GPU tensor is just like CPU tensors, there really is no difference. You cannot mix and match, though. Almost all algorithms require operands to be on the same device or to be moved there first. When you run the cell below, you should see the first addition print out a tensor filled with '2', then get an exception from the second addition.

In [ ]:
var a = torch.ones(3,4, device: torch.CUDA);
var b = torch.ones(3,4, device: torch.CUDA);
var c = torch.ones(3,4, device: torch.CPU);

(a + b).print();
(a + c).print();

Copying a tensor from device to device is very straight-forward. Note that applying 'to()' to a tensor does not move it. Because of this inefficiency, it is better to create a tensor where it belongs, than to first create it on the CPU and move it over later. That said, it is often necessary to copy input data, because it comes from files and in-memory data that can only be turned into tensors on a CPU.

There are two primary ways to copy a tensor to the GPU -- using 'cuda()' or using 'to()' The former is simpler, while the latter is more flexible -- it allows you to simultaneously convert the element type of a tensor and copy it over.

In [ ]:
(a + c.cuda()).print();
c.print();

// or:

(a + c.to(device:torch.CUDA)).print();
c.print();

## GPU Memory Management

Before moving on, it is important to discuss explicit memory management of tensors. Because CUDA does not have any virtual memory mechanism, it is easy to run out of GPU memory unless it is carefully managed.

TorchSharp tensors are eventually garbage-collected, which is triggered when the heap is starting to get full. However, the heap is all CPU memory, and only what is seen by the managed runtime. The storage for tensors are allocated in native code and doesn't therefore increase the memory pressure that triggers GC. This is particularly precarious for GPU memory.

Therefore, the tensor class implements IDisposable, so that you can manually free memory.

TorchSharp arithmetic results in a lot of temporaries, which need to be freed when no longer used. Consider this expression:

In [ ]:
(a + b) * (a + c.cuda())

There are three temporaries here, none of which are seeing their native memory freed. To demonstrate, let's pull all the temporaries out into explicit variables:

In [ ]:
var t0 = a + b;
var t1 = c.cuda();
var t2 = a + t1;
var t3 = t0 * t2;

t3

To deal with this systematically, we need to use 'using var' for all the temporaries. It's ugly, but necessary in order to efficiently manage GPU memory.

In [ ]:
using (torch.Tensor t0 = a + b, t1 = c.cuda(), t2 = a + t1 ) {

    var t3 = t0 * t2;
    t3.print();
}


### DisposeScope

In version 0.95.4, TorchSharp introduced the notion of a `DisposeScope`, which deals with the dispose pattern systematically. It introduces the notion of a dynamic (runtime) scope, which controls the liftime of all tensors created while the scope is in effect. The lexical scope, i.e. the source code location where variables are held, etc., had no impact on the dynamic scope management.

When any .NET tensor is created, it will be registered with the current dynamic scope, if there is one. Once registered, the tensor will be disposed automatically when the scope is disposed. It doesn't matter if the tensor is held in a variable declared outside or inside the scope.

Try running the next cell with and without the 'using' line (comment it out), and notice how the tensor count grows if you don't have it, but stays constant when you do.

In [ ]:
Console.WriteLine(torch.Tensor.TotalCount);
using (var d = torch.NewDisposeScope()) 
{
    var t3 = (a + b) * (a + c.cuda());
    t3.print();
}
Console.WriteLine(torch.Tensor.TotalCount);

Sometimes, you really need a tensor instance to survive the end of the dynamic scope. It can be detached from the scope, or moved to a surrounding scope (they nest). If there is no surrounding scope, it's the same thing.

For example, let's try this example -- you should get an exception that complains about an invalid tensor. It's because it was disposed before calling `print()`.

In [ ]:
torch.Tensor t3;
using (var d = torch.NewDisposeScope()) 
{
    t3 = (a + b) * (a + c.cuda());
}
t3.print();

The solution is to detach it before the end of the scope. Once it's been moved outside, the tensor needs to either land in an outer scope that will automatically dispose it, or it needs to be disposed explicitly in order to free native memory (unless waiting until GC kicks in is acceptable).

In [ ]:
torch.Tensor t3;
using (var d = torch.NewDisposeScope()) 
{
    t3 = d.MoveToOuter((a + b) * (a + c.cuda()));
}
t3.print();
Console.WriteLine(t3.IsInvalid);
t3.Dispose();
Console.WriteLine(t3.IsInvalid);

In [ ]:
torch.Tensor t3;
using (var d0 = torch.NewDisposeScope()) 
{
    using (var d1 = torch.NewDisposeScope()) 
    {
        t3 = d1.MoveToOuter((a + b) * (a + c.cuda()));
    }
    t3.print();
    Console.WriteLine(t3.IsInvalid);
}
Console.WriteLine(t3.IsInvalid);

# Placing Model Parameters on the GPU

To use a GPU, tensors have to be copied or moved there. When you train, your data preparation logic is responsible for getting data to the GPU, but we also need the weights there. TorchSharp supports this by defining a 'to()' method on Modules, which can be used to move (not copy) the weights the model relies on to the GPU (or back to the CPU). We haven't looked at models yet, but keep this in mind for later:

```C#
var model = ...;
model.to(torch.CUDA);
```